In [44]:
import pandas as pd


df_wdc_co2 = pd.read_excel('data_for_plots.xlsx', sheet_name='WDC CO2', skiprows=1)
df_ngrip_d18o = pd.read_excel('data_for_plots.xlsx', sheet_name='NGRIP d18O', skiprows=1)
df_wdc_d18o = pd.read_excel('data_for_plots.xlsx', sheet_name='WDC d18O', skiprows=3)
df_do= pd.read_excel('data_for_plots.xlsx', sheet_name='DO', skiprows=1)

# df_wdc_co2.head() 
# df_ngrip_d18o.head()
# df_wdc_d18o.head()

# remove nan values for all three dataframes
df_wdc_co2 = df_wdc_co2.dropna()
df_ngrip_d18o = df_ngrip_d18o.dropna()
df_wdc_d18o = df_wdc_d18o.dropna()

# rename columns to Age,Value
df_wdc_co2.columns = ['Age', 'Value', 'Error']
df_ngrip_d18o.columns = ['Age', 'Value']
df_wdc_d18o.columns = ['Age', 'Value']

# drop values in the second column of df_wdc_d18o that above 999
df_wdc_d18o = df_wdc_d18o[df_wdc_d18o.iloc[:, 1] < 999]

# Age *-1
df_wdc_co2.Age = df_wdc_co2.Age * -1
df_ngrip_d18o.Age = df_ngrip_d18o.Age * -1
df_wdc_d18o.Age = df_wdc_d18o.Age * -1

In [53]:
from bokeh.plotting import figure, show, output_file
from bokeh.layouts import column, row
from bokeh.models import CheckboxButtonGroup, CustomJS, HoverTool, ColumnDataSource, Div, Range1d, RangeSlider, TextInput, Button
from bokeh.io import curdoc
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

def interpolate_and_smooth(df, resolution=10, smooth_width=100):
    interp_func = interp1d(df.iloc[:, 0], df.iloc[:, 1], kind='linear', bounds_error=False)
    interpolated_ages = np.arange(df.iloc[:, 0].min(), df.iloc[:, 0].max() + resolution, resolution)
    interpolated_values = interp_func(interpolated_ages)
    window_size = smooth_width // resolution
    smoothed_values = np.convolve(interpolated_values, np.ones(window_size)/window_size, mode='valid')
    adjusted_ages = interpolated_ages[len(interpolated_ages) - len(smoothed_values):]
    return adjusted_ages, smoothed_values

# Interpolating and smoothing for δ18O data
wdc_d18o_ages, wdc_d18o_smoothed = interpolate_and_smooth(df_wdc_d18o)
ngrip_d18o_ages, ngrip_d18o_smoothed = interpolate_and_smooth(df_ngrip_d18o)

# Create a dictionary of data series
data_series = {
    "WDC CO2": df_wdc_co2,
    "WDC δ18O": df_wdc_d18o,
    "NGRIP δ18O": df_ngrip_d18o
}

wdc_d18o_ages=wdc_d18o_ages/1000
ngrip_d18o_ages=ngrip_d18o_ages/1000

# Create Bokeh plot for each series
plots = []
xlabels=['WD2014 Age (kyr, relative to AD 1950)', 'WD2014 Age (kyr, relative to AD 1950)', 'GICC05 Age (kyr, relative to AD 2000)']
ylabels=['CO2 (ppm)', 'δ18O (per mil)', 'δ18O (per mil)']
y_ranges=[(180, 290), (-44, -30), (-47, -32)]
y_range_middles=0.5*(np.array(y_ranges)[:,0]+np.array(y_ranges)[:,1])
y_hight=(np.array(y_ranges)[:,1]-np.array(y_ranges)[:,0])
DO_strips_dict = {}
DO_labels_dict = {}



for i, (name, df) in enumerate(data_series.items()):
    
    # if "DO" in name:
    #     continue

    source = ColumnDataSource(data=dict(x=df.iloc[:, 0]/1000, y=df.iloc[:, 1]))
    p = figure(sizing_mode='stretch_both', title=name)
    p.y_range = Range1d(*y_ranges[i])
    
    # Plot GS
    x = df_do['middle of GS'].values*-1/1000
    y = y_range_middles[i]*np.ones(len(x))
    width = df_do['GS duration'].values/1000
    height = y_hight[i]*np.ones(len(x))

    DO_strips=p.rect(x, 
            y, 
            width, 
            height,fill_color="#cab2d6")
    DO_strips.level = 'underlay' 
    DO_strips.visible = False
    DO_strips_dict[name] = DO_strips

    # add text annotation on the middle of GS, with 90 degree rotation
    # p.text(x=x+width/2, y=y, text=df_do['GS'].values, text_align='center', text_baseline='middle', text_font_size='8pt', angle=np.pi/2)
    DO_labels=p.text(x=df_do['middle of GS'].values*-1/1000, 
            y=(y_range_middles[i]+0.34*y_hight[i])*np.ones(len(x)),
            text=df_do['event'].values,
            text_font_size='8pt',
            text_align='center',
            text_baseline='middle',
            text_color='black',
            angle=np.pi/2
            
            )
    # set the order to be just above the strips
    DO_labels.level = 'glyph'
    DO_labels.visible = False
    DO_labels_dict[name] = DO_labels
    
    p.line('x', 'y', source=source, line_width=2)
    p.visible = False
    p.xaxis.axis_label = xlabels[i]
    p.yaxis.axis_label = ylabels[i]
    p.x_range = Range1d(-70, -10)
    hover = HoverTool(tooltips=[("Time", "@x"), ("Value", "@y")])
    p.add_tools(hover)

    if "δ18O" in name:
        if "WDC" in name:
            smoothed_source = ColumnDataSource(data=dict(x=wdc_d18o_ages, y=wdc_d18o_smoothed))
        elif "NGRIP" in name:
            smoothed_source = ColumnDataSource(data=dict(x=ngrip_d18o_ages, y=ngrip_d18o_smoothed))
        p.line('x', 'y', source=smoothed_source, line_width=2, color="red", legend_label="100 yr moving average")
        p.legend.location = "top_left"

    plots.append(p)

# Set the CO2 plot to be visible initially
plots[0].visible = True

# Create RangeSlider widget for x-axis range
x_range_slider = RangeSlider(start=-70, end=-10, value=(-70, -10), step=0.01, title="Age Range Fine-tuning", direction='ltr', width=500)

# Create TextInput widgets for min and max data range
min_range_input = TextInput(value="-70", title="Age Oldest:")
max_range_input = TextInput(value="-10", title="Age Youngest:")

# Create a 'Set' Button
set_button = Button(label="Set", button_type="success", width=100, height=65)

# Callback for the 'Set' Button
set_button_callback = CustomJS(args=dict(min_range_input=min_range_input, max_range_input=max_range_input, x_range_slider=x_range_slider), code="""
    const min_range = parseFloat(min_range_input.value);
    const max_range = parseFloat(max_range_input.value);
    if (min_range < max_range) {
        x_range_slider.start = min_range;
        x_range_slider.end = max_range;
        x_range_slider.value = [min_range, max_range];
                               

        p.x_range.start = min_range;
        p.x_range.end = max_range;
                               
    } else {
        alert('Min range should be less than max range.');
    }
""")
set_button.js_on_click(set_button_callback)

# Callback for the RangeSlider
slider_callback = CustomJS(args=dict(plots=plots, x_range_slider=x_range_slider), code="""
    const start = x_range_slider.value[0];
    const end = x_range_slider.value[1];
    for (let p of plots) {
        p.x_range.start = start;
        p.x_range.end = end;
    }
""")
x_range_slider.js_on_change('value', slider_callback)

checkbox_buttons = CheckboxButtonGroup(labels=list(data_series.keys()) + ["Greenland Stadials"], active=[0], sizing_mode='scale_width')
checkbox_callback = CustomJS(args=dict(plots=plots, DO_strips_dict=DO_strips_dict, DO_labels_dict=DO_labels_dict, checkbox_buttons=checkbox_buttons), code="""
    for (let i = 0; i < plots.length; i++) {  // Exclude the last button which is for Greenland Stadials
        plots[i].visible = this.active.includes(i);
    }
    // Handle Greenland Stadials toggle separately
    let gsIndex = checkbox_buttons.labels.length-1;
    if (checkbox_buttons.active.includes(gsIndex)) {
        for (let key in DO_strips_dict) {
            DO_strips_dict[key].visible = true;
            DO_labels_dict[key].visible = true;
        }
    } else {
        for (let key in DO_strips_dict) {
            DO_strips_dict[key].visible = false;
            DO_labels_dict[key].visible = false;
        }
    }

""")

checkbox_buttons.js_on_change('active', checkbox_callback)



checkbox_buttons.js_on_change('active', checkbox_callback)

# Arrange layout
range_controls = row(min_range_input, max_range_input, set_button)
# Create a Div to display reference text
# reference_text = Div(text="<i>Reference:<br>1. WDC CO<sub>2</sub>: Bauska et al., (2021), DOI: 10.1038/s41561-020-00680-2; Marcott et al., (2014), DOI: 10.1038/nature13799<br>2. WDC δ<sup>18</sup>O: WAIS, (2015), DOI: 10.1038/nature14401<br>3. NGRIP δ<sup>18</sup>O: Seierstad, et al. (2014), DOI: 10.1016/j.quascirev.2014.10.032; Rasmussen et al., (2014), DOI: 10.1016/j.quascirev.2014.09.007</i>", width=1000, height=30)
reference_text = Div(
    text="""<i>Developed by P. Zheng
    <br> Data Reference:
    <br>1. WDC CO<sub>2</sub>: Bauska et al., (2021), DOI: 10.1038/s41561-020-00680-2; 
    Marcott et al., (2014), DOI: 10.1038/nature13799
    <br>2. WDC δ<sup>18</sup>O: WAIS, (2015), DOI: 10.1038/nature14401
    <br>3. NGRIP δ<sup>18</sup>O: Seierstad, et al. (2014), DOI: 10.1016/j.quascirev.2014.10.032; 
    Rasmussen et al., (2014), DOI: 10.1016/j.quascirev.2014.09.007
    <br>4. Greenland Stadial timings are on GICC05 time scale (relative to AD 2000), Ref: Rasmussen et al., (2014), DOI: 10.1016/j.quascirev.2014.09.007
    </i>""",
    width=1000, 
    height=60  # Increased height to accommodate multiple lines
)

# range_controls = row(min_range_input, max_range_input, set_button, align="end")
layout = column(checkbox_buttons, range_controls, x_range_slider, *plots, reference_text,sizing_mode='stretch_both')
curdoc().add_root(layout)
output_file("interactive_plot_responsive.html")
show(layout)


# Add DO button

In [7]:
from bokeh.plotting import figure, show, output_file
from bokeh.layouts import column, row
from bokeh.models import CheckboxButtonGroup, CustomJS, HoverTool, ColumnDataSource, Div, Range1d, RangeSlider, TextInput, Button
from bokeh.io import curdoc
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

def interpolate_and_smooth(df, resolution=10, smooth_width=100):
    interp_func = interp1d(df.iloc[:, 0], df.iloc[:, 1], kind='linear', bounds_error=False)
    interpolated_ages = np.arange(df.iloc[:, 0].min(), df.iloc[:, 0].max() + resolution, resolution)
    interpolated_values = interp_func(interpolated_ages)
    window_size = smooth_width // resolution
    smoothed_values = np.convolve(interpolated_values, np.ones(window_size)/window_size, mode='valid')
    adjusted_ages = interpolated_ages[len(interpolated_ages) - len(smoothed_values):]
    return adjusted_ages, smoothed_values

# Interpolating and smoothing for δ18O data
wdc_d18o_ages, wdc_d18o_smoothed = interpolate_and_smooth(df_wdc_d18o)
ngrip_d18o_ages, ngrip_d18o_smoothed = interpolate_and_smooth(df_ngrip_d18o)

# Create a dictionary of data series
data_series = {
    "WDC CO2": df_wdc_co2,
    "WDC δ18O": df_wdc_d18o,
    "NGRIP δ18O": df_ngrip_d18o,
    "DO events": df_do
}

wdc_d18o_ages=wdc_d18o_ages/1000
ngrip_d18o_ages=ngrip_d18o_ages/1000

# Create Bokeh plot for each series
plots = []
xlabels=['WD2014 Age (kyr, relative to AD 1950)', 'WD2014 Age (kyr, relative to AD 1950)', 'GICC05 Age (kyr, relative to AD 2000)']
ylabels=['CO2 (ppm)', 'δ18O (per mil)', 'δ18O (per mil)']

for i, (name, df) in enumerate(data_series.items()):
    
    if "DO" in name:
        continue

    source = ColumnDataSource(data=dict(x=df.iloc[:, 0]/1000, y=df.iloc[:, 1]))
    p = figure(sizing_mode='stretch_both', title=name)
    p.line('x', 'y', source=source, line_width=2)
    p.visible = False
    p.xaxis.axis_label = xlabels[i]
    p.yaxis.axis_label = ylabels[i]
    p.x_range = Range1d(-70, -10)
    hover = HoverTool(tooltips=[("Time", "@x"), ("Value", "@y")])
    p.add_tools(hover)

    if "δ18O" in name:
        if "WDC" in name:
            smoothed_source = ColumnDataSource(data=dict(x=wdc_d18o_ages, y=wdc_d18o_smoothed))
        elif "NGRIP" in name:
            smoothed_source = ColumnDataSource(data=dict(x=ngrip_d18o_ages, y=ngrip_d18o_smoothed))
        p.line('x', 'y', source=smoothed_source, line_width=2, color="red", legend_label="100 yr moving average")
        p.legend.location = "top_left"

    plots.append(p)

# Set the CO2 plot to be visible initially
plots[0].visible = True

# Create RangeSlider widget for x-axis range
x_range_slider = RangeSlider(start=-70, end=-10, value=(-70, -10), step=0.01, title="Age Range Fine-tuning", direction='ltr', width=500)

# Create TextInput widgets for min and max data range
min_range_input = TextInput(value="-70", title="Age Oldest:")
max_range_input = TextInput(value="-10", title="Age Youngest:")

# Create a 'Set' Button
set_button = Button(label="Set", button_type="success", width=100, height=65)

# Callback for the 'Set' Button
set_button_callback = CustomJS(args=dict(min_range_input=min_range_input, max_range_input=max_range_input, x_range_slider=x_range_slider), code="""
    const min_range = parseFloat(min_range_input.value);
    const max_range = parseFloat(max_range_input.value);
    if (min_range < max_range) {
        x_range_slider.start = min_range;
        x_range_slider.end = max_range;
        x_range_slider.value = [min_range, max_range];
                               

        p.x_range.start = min_range;
        p.x_range.end = max_range;
                               
    } else {
        alert('Min range should be less than max range.');
    }
""")
set_button.js_on_click(set_button_callback)

# Callback for the RangeSlider
slider_callback = CustomJS(args=dict(plots=plots, x_range_slider=x_range_slider), code="""
    const start = x_range_slider.value[0];
    const end = x_range_slider.value[1];
    for (let p of plots) {
        p.x_range.start = start;
        p.x_range.end = end;
    }
""")
x_range_slider.js_on_change('value', slider_callback)

# CheckboxButtonGroup and its callback
checkbox_buttons = CheckboxButtonGroup(labels=list(data_series.keys()), active=[0], sizing_mode='scale_width')
checkbox_callback = CustomJS(args=dict(plots=plots), code="""
    for (let i = 0; i < plots.length; i++) {
        plots[i].visible = this.active.includes(i);
    }
""")
checkbox_buttons.js_on_change('active', checkbox_callback)

# Arrange layout
range_controls = row(min_range_input, max_range_input, set_button)
# Create a Div to display reference text
# reference_text = Div(text="<i>Reference:<br>1. WDC CO<sub>2</sub>: Bauska et al., (2021), DOI: 10.1038/s41561-020-00680-2; Marcott et al., (2014), DOI: 10.1038/nature13799<br>2. WDC δ<sup>18</sup>O: WAIS, (2015), DOI: 10.1038/nature14401<br>3. NGRIP δ<sup>18</sup>O: Seierstad, et al. (2014), DOI: 10.1016/j.quascirev.2014.10.032; Rasmussen et al., (2014), DOI: 10.1016/j.quascirev.2014.09.007</i>", width=1000, height=30)
reference_text = Div(
    text="""<i>Developed by P. Zheng
    <br> Data Reference:
    <br>1. WDC CO<sub>2</sub>: Bauska et al., (2021), DOI: 10.1038/s41561-020-00680-2; 
    Marcott et al., (2014), DOI: 10.1038/nature13799
    <br>2. WDC δ<sup>18</sup>O: WAIS, (2015), DOI: 10.1038/nature14401
    <br>3. NGRIP δ<sup>18</sup>O: Seierstad, et al. (2014), DOI: 10.1016/j.quascirev.2014.10.032; 
    Rasmussen et al., (2014), DOI: 10.1016/j.quascirev.2014.09.007</i>""",
    width=1000, 
    height=60  # Increased height to accommodate multiple lines
)

# range_controls = row(min_range_input, max_range_input, set_button, align="end")
layout = column(checkbox_buttons, range_controls, x_range_slider, *plots, reference_text,sizing_mode='stretch_both')
curdoc().add_root(layout)
output_file("interactive_plot_responsive.html")
show(layout)


In [6]:
from bokeh.plotting import figure, show, output_file
from bokeh.layouts import column, row
from bokeh.models import CheckboxButtonGroup, CustomJS, HoverTool, ColumnDataSource, Div, Range1d, RangeSlider, TextInput, Button
from bokeh.io import curdoc
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

def interpolate_and_smooth(df, resolution=10, smooth_width=100):
    interp_func = interp1d(df.iloc[:, 0], df.iloc[:, 1], kind='linear', bounds_error=False)
    interpolated_ages = np.arange(df.iloc[:, 0].min(), df.iloc[:, 0].max() + resolution, resolution)
    interpolated_values = interp_func(interpolated_ages)
    window_size = smooth_width // resolution
    smoothed_values = np.convolve(interpolated_values, np.ones(window_size)/window_size, mode='valid')
    adjusted_ages = interpolated_ages[len(interpolated_ages) - len(smoothed_values):]
    return adjusted_ages, smoothed_values

# Interpolating and smoothing for δ18O data
wdc_d18o_ages, wdc_d18o_smoothed = interpolate_and_smooth(df_wdc_d18o)
ngrip_d18o_ages, ngrip_d18o_smoothed = interpolate_and_smooth(df_ngrip_d18o)

# Create a dictionary of data series
data_series = {
    "WDC CO2": df_wdc_co2,
    "WDC δ18O": df_wdc_d18o,
    "NGRIP δ18O": df_ngrip_d18o
}

wdc_d18o_ages=wdc_d18o_ages/1000
ngrip_d18o_ages=ngrip_d18o_ages/1000

# Create Bokeh plot for each series
plots = []
xlabels=['WD2014 Age (kyr, relative to AD 1950)', 'WD2014 Age (kyr, relative to AD 1950)', 'GICC05 Age (kyr, relative to AD 2000)']
ylabels=['CO2 (ppm)', 'δ18O (per mil)', 'δ18O (per mil)']

for i, (name, df) in enumerate(data_series.items()):
    source = ColumnDataSource(data=dict(x=df.iloc[:, 0]/1000, y=df.iloc[:, 1]))
    p = figure(sizing_mode='stretch_both', title=name)
    p.line('x', 'y', source=source, line_width=2)
    p.visible = False
    p.xaxis.axis_label = xlabels[i]
    p.yaxis.axis_label = ylabels[i]
    p.x_range = Range1d(-70, -10)
    hover = HoverTool(tooltips=[("Time", "@x"), ("Value", "@y")])
    p.add_tools(hover)

    if "δ18O" in name:
        if "WDC" in name:
            smoothed_source = ColumnDataSource(data=dict(x=wdc_d18o_ages, y=wdc_d18o_smoothed))
        elif "NGRIP" in name:
            smoothed_source = ColumnDataSource(data=dict(x=ngrip_d18o_ages, y=ngrip_d18o_smoothed))
        p.line('x', 'y', source=smoothed_source, line_width=2, color="red", legend_label="100 yr moving average")
        p.legend.location = "top_left"

    plots.append(p)

# Set the CO2 plot to be visible initially
plots[0].visible = True

# Create RangeSlider widget for x-axis range
x_range_slider = RangeSlider(start=-70, end=-10, value=(-70, -10), step=0.01, title="Age Range Fine-tuning", direction='ltr', width=500)

# Create TextInput widgets for min and max data range
min_range_input = TextInput(value="-70", title="Age Oldest:")
max_range_input = TextInput(value="-10", title="Age Youngest:")

# Create a 'Set' Button
set_button = Button(label="Set", button_type="success", width=100, height=65)

# Callback for the 'Set' Button
set_button_callback = CustomJS(args=dict(min_range_input=min_range_input, max_range_input=max_range_input, x_range_slider=x_range_slider), code="""
    const min_range = parseFloat(min_range_input.value);
    const max_range = parseFloat(max_range_input.value);
    if (min_range < max_range) {
        x_range_slider.start = min_range;
        x_range_slider.end = max_range;
        x_range_slider.value = [min_range, max_range];
                               

        p.x_range.start = min_range;
        p.x_range.end = max_range;
                               
    } else {
        alert('Min range should be less than max range.');
    }
""")
set_button.js_on_click(set_button_callback)

# Callback for the RangeSlider
slider_callback = CustomJS(args=dict(plots=plots, x_range_slider=x_range_slider), code="""
    const start = x_range_slider.value[0];
    const end = x_range_slider.value[1];
    for (let p of plots) {
        p.x_range.start = start;
        p.x_range.end = end;
    }
""")
x_range_slider.js_on_change('value', slider_callback)

# CheckboxButtonGroup and its callback
checkbox_buttons = CheckboxButtonGroup(labels=list(data_series.keys()), active=[0], sizing_mode='scale_width')
checkbox_callback = CustomJS(args=dict(plots=plots), code="""
    for (let i = 0; i < plots.length; i++) {
        plots[i].visible = this.active.includes(i);
    }
""")
checkbox_buttons.js_on_change('active', checkbox_callback)

# Arrange layout
range_controls = row(min_range_input, max_range_input, set_button)
# Create a Div to display reference text
# reference_text = Div(text="<i>Reference:<br>1. WDC CO<sub>2</sub>: Bauska et al., (2021), DOI: 10.1038/s41561-020-00680-2; Marcott et al., (2014), DOI: 10.1038/nature13799<br>2. WDC δ<sup>18</sup>O: WAIS, (2015), DOI: 10.1038/nature14401<br>3. NGRIP δ<sup>18</sup>O: Seierstad, et al. (2014), DOI: 10.1016/j.quascirev.2014.10.032; Rasmussen et al., (2014), DOI: 10.1016/j.quascirev.2014.09.007</i>", width=1000, height=30)
reference_text = Div(
    text="""<i>Developed by P. Zheng
    <br> Data Reference:
    <br>1. WDC CO<sub>2</sub>: Bauska et al., (2021), DOI: 10.1038/s41561-020-00680-2; 
    Marcott et al., (2014), DOI: 10.1038/nature13799
    <br>2. WDC δ<sup>18</sup>O: WAIS, (2015), DOI: 10.1038/nature14401
    <br>3. NGRIP δ<sup>18</sup>O: Seierstad, et al. (2014), DOI: 10.1016/j.quascirev.2014.10.032; 
    Rasmussen et al., (2014), DOI: 10.1016/j.quascirev.2014.09.007</i>""",
    width=1000, 
    height=60  # Increased height to accommodate multiple lines
)

# range_controls = row(min_range_input, max_range_input, set_button, align="end")
layout = column(checkbox_buttons, range_controls, x_range_slider, *plots, reference_text,sizing_mode='stretch_both')
curdoc().add_root(layout)
output_file("interactive_plot_responsive.html")
show(layout)


In [8]:
from bokeh.plotting import figure, show, output_file
from bokeh.layouts import column, row
from bokeh.models import CheckboxButtonGroup, CustomJS, HoverTool, ColumnDataSource, Div, Range1d, RangeSlider
from bokeh.io import curdoc
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

# Assuming df_wdc_co2, df_wdc_d18o, and df_ngrip_d18o are defined Pandas DataFrames

def interpolate_and_smooth(df, resolution=10, smooth_width=100):
    interp_func = interp1d(df.iloc[:, 0], df.iloc[:, 1], kind='linear', bounds_error=False)
    interpolated_ages = np.arange(df.iloc[:, 0].min(), df.iloc[:, 0].max() + resolution, resolution)
    interpolated_values = interp_func(interpolated_ages)
    window_size = smooth_width // resolution
    smoothed_values = np.convolve(interpolated_values, np.ones(window_size)/window_size, mode='valid')
    adjusted_ages = interpolated_ages[len(interpolated_ages) - len(smoothed_values):]
    return adjusted_ages, smoothed_values

# Interpolating and smoothing for δ18O data
wdc_d18o_ages, wdc_d18o_smoothed = interpolate_and_smooth(df_wdc_d18o)
ngrip_d18o_ages, ngrip_d18o_smoothed = interpolate_and_smooth(df_ngrip_d18o)

# Create a dictionary of data series
data_series = {
    "WDC CO2": df_wdc_co2,
    "WDC δ18O": df_wdc_d18o,
    "NGRIP δ18O": df_ngrip_d18o
}

# Create Bokeh plot for each series
plots = []
xlabels=['WD2014 Age (yr, relative to AD 1950)', 'WD2014 Age (yr, relative to AD 1950)', 'GICC05 Age (yr, relative to AD 2000)']
ylabels=['CO2 (ppm)', 'δ18O (per mil)', 'δ18O (per mil)']

for i, (name, df) in enumerate(data_series.items()):
    source = ColumnDataSource(data=dict(x=df.iloc[:, 0], y=df.iloc[:, 1]))
    p = figure(sizing_mode='stretch_both', title=name)
    p.line('x', 'y', source=source, line_width=2)
    p.visible = False
    p.xaxis.axis_label = xlabels[i]
    p.yaxis.axis_label = ylabels[i]
    # p.x_range.flipped = True
    # p.x_range = Range1d(70000, 0)
    p.x_range = Range1d(-122500, 0)
    hover = HoverTool(tooltips=[("Time", "@x"), ("Value", "@y")])
    p.add_tools(hover)

    if "δ18O" in name:
        if "WDC" in name:
            smoothed_source = ColumnDataSource(data=dict(x=wdc_d18o_ages, y=wdc_d18o_smoothed))
        elif "NGRIP" in name:
            smoothed_source = ColumnDataSource(data=dict(x=ngrip_d18o_ages, y=ngrip_d18o_smoothed))
        p.line('x', 'y', source=smoothed_source, line_width=2, color="red", legend_label="100 yr moving average")
        p.legend.location = "top_left"

    plots.append(p)

# Set the CO2 plot to be visible initially
plots[0].visible = True

# # Create RangeSlider widget for x-axis range
# x_range_slider = RangeSlider(start=0, end=70000, value=(0, 70000), step=10, title="Age range", direction='ltr', width=500)

x_range_slider = RangeSlider(start=-122500, end=0, value=(-122500, 0), step=10, title="X Range", direction='ltr', width=500)

# Callback for the slider
callback = CustomJS(args=dict(plots=plots, x_range_slider=x_range_slider), code="""
    const start = x_range_slider.value[0];
    const end = x_range_slider.value[1];
    for (let p of plots) {
        p.x_range.start = start;
        p.x_range.end = end;
    }
""")
x_range_slider.js_on_change('value', callback)

checkbox_buttons = CheckboxButtonGroup(labels=list(data_series.keys()), active=[0], sizing_mode='scale_width')
checkbox_callback = CustomJS(args=dict(plots=plots), code="""
    for (let i = 0; i < plots.length; i++) {
        plots[i].visible = this.active.includes(i);
    }
""")
checkbox_buttons.js_on_change('active', checkbox_callback)

# Arrange plots, slider, and CheckboxButtonGroup in a column layout
layout = column(checkbox_buttons, x_range_slider, *plots, sizing_mode='stretch_both')
curdoc().add_root(layout)
output_file("PastClimatePlots/index.html.html")
show(layout)


In [4]:
from bokeh.plotting import figure, show, output_file
from bokeh.layouts import column, row
from bokeh.models import CheckboxButtonGroup, CustomJS, HoverTool, ColumnDataSource, Div, Range1d, RangeSlider
from bokeh.io import curdoc
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

# Assuming df_wdc_co2, df_wdc_d18o, and df_ngrip_d18o are defined Pandas DataFrames

def interpolate_and_smooth(df, resolution=10, smooth_width=100):
    interp_func = interp1d(df.iloc[:, 0], df.iloc[:, 1], kind='linear', bounds_error=False)
    interpolated_ages = np.arange(df.iloc[:, 0].min(), df.iloc[:, 0].max() + resolution, resolution)
    interpolated_values = interp_func(interpolated_ages)
    window_size = smooth_width // resolution
    smoothed_values = np.convolve(interpolated_values, np.ones(window_size)/window_size, mode='valid')
    adjusted_ages = interpolated_ages[len(interpolated_ages) - len(smoothed_values):]
    return adjusted_ages, smoothed_values

# Interpolating and smoothing for δ18O data
wdc_d18o_ages, wdc_d18o_smoothed = interpolate_and_smooth(df_wdc_d18o)
ngrip_d18o_ages, ngrip_d18o_smoothed = interpolate_and_smooth(df_ngrip_d18o)

# Create a dictionary of data series
data_series = {
    "WDC CO2": df_wdc_co2,
    "WDC δ18O": df_wdc_d18o,
    "NGRIP δ18O": df_ngrip_d18o
}

# Create Bokeh plot for each series
plots = []
xlabels=['WD2014 Age (yr, relative to AD 1950)', 'WD2014 Age (yr, relative to AD 1950)', 'GICC05 Age (yr, relative to AD 2000)']
ylabels=['CO2 (ppm)', 'δ18O (per mil)', 'δ18O (per mil)']

for i, (name, df) in enumerate(data_series.items()):
    source = ColumnDataSource(data=dict(x=df.iloc[:, 0], y=df.iloc[:, 1]))
    p = figure(sizing_mode='stretch_both', title=name)
    p.line('x', 'y', source=source, line_width=2)
    p.visible = False
    p.xaxis.axis_label = xlabels[i]
    p.yaxis.axis_label = ylabels[i]
    # p.x_range.flipped = True
    # p.x_range = Range1d(70000, 0)
    p.x_range = Range1d(-70000, 0)
    hover = HoverTool(tooltips=[("Time", "@x"), ("Value", "@y")])
    p.add_tools(hover)

    if "δ18O" in name:
        if "WDC" in name:
            smoothed_source = ColumnDataSource(data=dict(x=wdc_d18o_ages, y=wdc_d18o_smoothed))
        elif "NGRIP" in name:
            smoothed_source = ColumnDataSource(data=dict(x=ngrip_d18o_ages, y=ngrip_d18o_smoothed))
        p.line('x', 'y', source=smoothed_source, line_width=2, color="red", legend_label="100 yr moving average")
        p.legend.location = "top_left"

    plots.append(p)

# Set the CO2 plot to be visible initially
plots[0].visible = True

# # Create RangeSlider widget for x-axis range
# x_range_slider = RangeSlider(start=0, end=70000, value=(0, 70000), step=10, title="Age range", direction='ltr', width=500)

x_range_slider = RangeSlider(start=-70000, end=0, value=(-70000, 0), step=10, title="X Range", direction='ltr', width=500)

# Callback for the slider
callback = CustomJS(args=dict(plots=plots, x_range_slider=x_range_slider), code="""
    const start = x_range_slider.value[0];
    const end = x_range_slider.value[1];
    for (let p of plots) {
        p.x_range.start = start;
        p.x_range.end = end;
    }
""")
x_range_slider.js_on_change('value', callback)

checkbox_buttons = CheckboxButtonGroup(labels=list(data_series.keys()), active=[0], sizing_mode='scale_width')
checkbox_callback = CustomJS(args=dict(plots=plots), code="""
    for (let i = 0; i < plots.length; i++) {
        plots[i].visible = this.active.includes(i);
    }
""")
checkbox_buttons.js_on_change('active', checkbox_callback)

# Arrange plots, slider, and CheckboxButtonGroup in a column layout
layout = column(checkbox_buttons, x_range_slider, *plots, sizing_mode='stretch_both')
curdoc().add_root(layout)
output_file("interactive_plot_responsive.html")
show(layout)


In [64]:
from bokeh.plotting import figure, show, output_file
from bokeh.layouts import column, row
from bokeh.models import CheckboxButtonGroup, CustomJS, HoverTool, ColumnDataSource, Div, Range1d, RangeSlider
from bokeh.io import curdoc
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

# Assuming df_wdc_co2, df_wdc_d18o, and df_ngrip_d18o are defined Pandas DataFrames

def interpolate_and_smooth(df, resolution=10, smooth_width=100):
    interp_func = interp1d(df.iloc[:, 0], df.iloc[:, 1], kind='linear', bounds_error=False)
    interpolated_ages = np.arange(df.iloc[:, 0].min(), df.iloc[:, 0].max() + resolution, resolution)
    interpolated_values = interp_func(interpolated_ages)
    window_size = smooth_width // resolution
    smoothed_values = np.convolve(interpolated_values, np.ones(window_size)/window_size, mode='valid')
    adjusted_ages = interpolated_ages[len(interpolated_ages) - len(smoothed_values):]
    return adjusted_ages, smoothed_values

# Interpolating and smoothing for δ18O data
wdc_d18o_ages, wdc_d18o_smoothed = interpolate_and_smooth(df_wdc_d18o)
ngrip_d18o_ages, ngrip_d18o_smoothed = interpolate_and_smooth(df_ngrip_d18o)

# Create a dictionary of data series
data_series = {
    "WDC CO2": df_wdc_co2,
    "WDC δ18O": df_wdc_d18o,
    "NGRIP δ18O": df_ngrip_d18o
}

# Create Bokeh plot for each series
plots = []
xlabels=['WD2014 Age (yr b1950)', 'WD2014 Age (yr b1950)', 'GICC05 Age (yr b2k)']
ylabels=['CO2 (ppm)', 'δ18O (per mil)', 'δ18O (per mil)']

for i, (name, df) in enumerate(data_series.items()):
    source = ColumnDataSource(data=dict(x=df.iloc[:, 0], y=df.iloc[:, 1]))
    p = figure(sizing_mode='stretch_both', title=name)
    p.line('x', 'y', source=source, line_width=2)
    p.visible = False
    p.xaxis.axis_label = xlabels[i]
    p.yaxis.axis_label = ylabels[i]
    # p.x_range.flipped = True
    # p.x_range = Range1d(70000, 0)
    p.x_range = Range1d(0, 70000)
    hover = HoverTool(tooltips=[("Time", "@x"), ("Value", "@y")])
    p.add_tools(hover)

    if "δ18O" in name:
        if "WDC" in name:
            smoothed_source = ColumnDataSource(data=dict(x=wdc_d18o_ages, y=wdc_d18o_smoothed))
        elif "NGRIP" in name:
            smoothed_source = ColumnDataSource(data=dict(x=ngrip_d18o_ages, y=ngrip_d18o_smoothed))
        p.line('x', 'y', source=smoothed_source, line_width=2, color="red", legend_label="100 yr moving average")
        # p.legend.location = "top_left"

    plots.append(p)

# Set the CO2 plot to be visible initially
plots[0].visible = True

# # Create RangeSlider widget for x-axis range
x_range_slider = RangeSlider(start=0, end=70000, value=(0, 70000), step=10, title="Age range", direction='ltr', width=500)

# x_range_slider = RangeSlider(start=70000, end=0, value=(70000, 0), step=-10, title="X Range", direction='rtl', width=500)

# Callback for the slider
callback = CustomJS(args=dict(plots=plots, x_range_slider=x_range_slider), code="""
    const start = x_range_slider.value[0];
    const end = x_range_slider.value[1];
    for (let p of plots) {
        p.x_range.start = start;
        p.x_range.end = end;
    }
""")
x_range_slider.js_on_change('value', callback)

checkbox_buttons = CheckboxButtonGroup(labels=list(data_series.keys()), active=[0], sizing_mode='scale_width')
checkbox_callback = CustomJS(args=dict(plots=plots), code="""
    for (let i = 0; i < plots.length; i++) {
        plots[i].visible = this.active.includes(i);
    }
""")
checkbox_buttons.js_on_change('active', checkbox_callback)

# Arrange plots, slider, and CheckboxButtonGroup in a column layout
layout = column(checkbox_buttons, x_range_slider, *plots, sizing_mode='stretch_both')
curdoc().add_root(layout)
output_file("interactive_plot_responsive.html")
show(layout)


In [58]:
from bokeh.plotting import figure, show, output_file
from bokeh.layouts import column, row
from bokeh.models import CheckboxButtonGroup, CustomJS, HoverTool, ColumnDataSource, Div, Range1d, RangeSlider
from bokeh.io import curdoc
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

# Assuming df_wdc_co2, df_wdc_d18o, and df_ngrip_d18o are defined Pandas DataFrames

def interpolate_and_smooth(df, resolution=10, smooth_width=100):
    interp_func = interp1d(df.iloc[:, 0], df.iloc[:, 1], kind='linear', bounds_error=False)
    interpolated_ages = np.arange(df.iloc[:, 0].min(), df.iloc[:, 0].max() + resolution, resolution)
    interpolated_values = interp_func(interpolated_ages)
    window_size = smooth_width // resolution
    smoothed_values = np.convolve(interpolated_values, np.ones(window_size)/window_size, mode='valid')
    adjusted_ages = interpolated_ages[len(interpolated_ages) - len(smoothed_values):]
    return adjusted_ages, smoothed_values

# Interpolating and smoothing for δ18O data
wdc_d18o_ages, wdc_d18o_smoothed = interpolate_and_smooth(df_wdc_d18o)
ngrip_d18o_ages, ngrip_d18o_smoothed = interpolate_and_smooth(df_ngrip_d18o)

# Create a dictionary of data series
data_series = {
    "WDC CO2": df_wdc_co2,
    "WDC δ18O": df_wdc_d18o,
    "NGRIP δ18O": df_ngrip_d18o
}

# Create Bokeh plot for each series
plots = []
xlabels=['WD2014 Age (yr b1950)', 'WD2014 Age (yr b1950)', 'GICC05 Age (yr b2k)']
ylabels=['CO2 (ppm)', 'δ18O (per mil)', 'δ18O (per mil)']

for i, (name, df) in enumerate(data_series.items()):
    source = ColumnDataSource(data=dict(x=df.iloc[:, 0], y=df.iloc[:, 1]))
    p = figure(sizing_mode='stretch_both', title=name)
    p.line('x', 'y', source=source, line_width=2)
    p.visible = False
    p.xaxis.axis_label = xlabels[i]
    p.yaxis.axis_label = ylabels[i]
    p.x_range.flipped = True
    p.x_range = Range1d(70000, 0)
    # p.x_range = Range1d(0, 70000)
    hover = HoverTool(tooltips=[("Time", "@x"), ("Value", "@y")])
    p.add_tools(hover)

    if "δ18O" in name:
        if "WDC" in name:
            smoothed_source = ColumnDataSource(data=dict(x=wdc_d18o_ages, y=wdc_d18o_smoothed))
        elif "NGRIP" in name:
            smoothed_source = ColumnDataSource(data=dict(x=ngrip_d18o_ages, y=ngrip_d18o_smoothed))
        p.line('x', 'y', source=smoothed_source, line_width=2, color="red", legend_label="100 yr moving average")
        p.legend.location = "top_left"

    plots.append(p)

# Set the CO2 plot to be visible initially
plots[0].visible = True

# # Create RangeSlider widget for x-axis range
x_range_slider = RangeSlider(start=0, end=70000, value=(0, 70000), step=10, title="X Range", direction='ltr', width=500)

# x_range_slider = RangeSlider(start=70000, end=0, value=(70000, 0), step=10, title="X Range", direction='rtl', width=500)

# Callback for the slider
callback = CustomJS(args=dict(plots=plots, x_range_slider=x_range_slider), code="""
    const start = x_range_slider.value[0];
    const end = x_range_slider.value[1];
    for (let p of plots) {
        p.x_range.start = end;
        p.x_range.end = start;
    }
""")
x_range_slider.js_on_change('value', callback)

checkbox_buttons = CheckboxButtonGroup(labels=list(data_series.keys()), active=[0], sizing_mode='scale_width')
checkbox_callback = CustomJS(args=dict(plots=plots), code="""
    for (let i = 0; i < plots.length; i++) {
        plots[i].visible = this.active.includes(i);
    }
""")
checkbox_buttons.js_on_change('active', checkbox_callback)

# Arrange plots, slider, and CheckboxButtonGroup in a column layout
layout = column(checkbox_buttons, x_range_slider, *plots, sizing_mode='stretch_both')
curdoc().add_root(layout)
output_file("interactive_plot_responsive.html")
show(layout)


In [45]:
from bokeh.plotting import figure, show, output_file
from bokeh.layouts import column, row
from bokeh.models import CheckboxButtonGroup, CustomJS, HoverTool, ColumnDataSource, Div, Range1d
from bokeh.io import curdoc
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d

# Assuming df_wdc_co2, df_wdc_d18o, and df_ngrip_d18o are defined Pandas DataFrames

def interpolate_and_smooth(df, resolution=10, smooth_width=100):
    interp_func = interp1d(df.iloc[:, 0], df.iloc[:, 1], kind='linear', bounds_error=False)
    interpolated_ages = np.arange(df.iloc[:, 0].min(), df.iloc[:, 0].max() + resolution, resolution)
    interpolated_values = interp_func(interpolated_ages)

    # Moving average smoothing
    window_size = smooth_width // resolution
    smoothed_values = np.convolve(interpolated_values, np.ones(window_size)/window_size, mode='valid')
    # Adjust ages to align with the center of the smoothing window
    adjusted_ages = interpolated_ages[len(interpolated_ages) - len(smoothed_values):]

    return adjusted_ages, smoothed_values

# Interpolating and smoothing for δ18O data
wdc_d18o_ages, wdc_d18o_smoothed = interpolate_and_smooth(df_wdc_d18o)
ngrip_d18o_ages, ngrip_d18o_smoothed = interpolate_and_smooth(df_ngrip_d18o)

# Create a dictionary of data series
data_series = {
    "WDC CO2": df_wdc_co2,
    "WDC δ18O": df_wdc_d18o,
    "NGRIP δ18O": df_ngrip_d18o
}

# Create Bokeh plot for each series
plots = []
xlabels=['WD2014 Age (yr b1950)', 'WD2014 Age (yr b1950)', 'GICC05 Age (yr b2k)']
ylabels=['CO2 (ppm)', 'δ18O (per mil)', 'δ18O (per mil)']

for i, (name, df) in enumerate(data_series.items()):
    source = ColumnDataSource(data=dict(x=df.iloc[:, 0], y=df.iloc[:, 1]))
    p = figure(sizing_mode='stretch_both', title=name)
    p.line('x', 'y', source=source, line_width=2)
    p.visible = False  # Initially hide the plots
    p.xaxis.axis_label = xlabels[i]
    p.yaxis.axis_label = ylabels[i]
    p.x_range.flipped = True
    p.x_range=Range1d(70000, 0)
    hover = HoverTool(tooltips=[("Time", "@x"), ("Value", "@y")])
    p.add_tools(hover)

    # Add smoothed curve for δ18O data
    if "δ18O" in name:
        if "WDC" in name:
            smoothed_source = ColumnDataSource(data=dict(x=wdc_d18o_ages, y=wdc_d18o_smoothed))
        elif "NGRIP" in name:
            smoothed_source = ColumnDataSource(data=dict(x=ngrip_d18o_ages, y=ngrip_d18o_smoothed))
        p.line('x', 'y', source=smoothed_source, line_width=2, color="red", legend_label="100 yr moving average")
        p.legend.location = "top_left"

    plots.append(p)

plots[0].visible = True

checkbox_buttons = CheckboxButtonGroup(labels=list(data_series.keys()), active=[0], sizing_mode='scale_width')
callback = CustomJS(args=dict(plots=plots), code="""
    for (let i = 0; i < plots.length; i++) {
        plots[i].visible = this.active.includes(i);
    }
""")
checkbox_buttons.js_on_change('active', callback)

layout = column(checkbox_buttons, *plots, sizing_mode='stretch_both')
curdoc().add_root(layout)
output_file("interactive_plot_responsive.html")
show(layout)


In [44]:
from bokeh.plotting import figure, show, output_file
from bokeh.layouts import column, row
from bokeh.models import CheckboxButtonGroup, CustomJS, HoverTool, ColumnDataSource, Div, Range1d
from bokeh.io import curdoc
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from scipy.ndimage import gaussian_filter1d

# Assuming df_wdc_co2, df_wdc_d18o, and df_ngrip_d18o are defined Pandas DataFrames

def interpolate_and_smooth(df, resolution=10, smooth_width=100):
    interp_func = interp1d(df.iloc[:, 0], df.iloc[:, 1], kind='linear', bounds_error=False)
    interpolated_ages = np.arange(df.iloc[:, 0].min(), df.iloc[:, 0].max() + resolution, resolution)
    interpolated_values = interp_func(interpolated_ages)
    sigma = smooth_width / resolution
    smoothed_values = gaussian_filter1d(interpolated_values, sigma=sigma)
    return interpolated_ages, smoothed_values

# Interpolating and smoothing for δ18O data
wdc_d18o_ages, wdc_d18o_smoothed = interpolate_and_smooth(df_wdc_d18o)
ngrip_d18o_ages, ngrip_d18o_smoothed = interpolate_and_smooth(df_ngrip_d18o)

# Create a dictionary of data series
data_series = {
    "WDC CO2": df_wdc_co2,
    "WDC δ18O": df_wdc_d18o,
    "NGRIP δ18O": df_ngrip_d18o
}

# Create Bokeh plot for each series
plots = []
xlabels=['WD2014 Age (yr b1950)', 'WD2014 Age (yr b1950)', 'GICC05 Age (yr b2k)']
ylabels=['CO2 (ppm)', 'δ18O (per mil)', 'δ18O (per mil)']

for i, (name, df) in enumerate(data_series.items()):
    source = ColumnDataSource(data=dict(x=df.iloc[:, 0], y=df.iloc[:, 1]))
    p = figure(sizing_mode='stretch_both', title=name)
    p.line('x', 'y', source=source, line_width=2)
    p.visible = False  # Initially hide the plots
    p.xaxis.axis_label = xlabels[i]
    p.yaxis.axis_label = ylabels[i]
    p.x_range.flipped = True
    p.x_range=Range1d(70000, 0)
    hover = HoverTool(tooltips=[("Time", "@x"), ("Value", "@y")])
    p.add_tools(hover)

    # Add smoothed curve for δ18O data
    if "δ18O" in name:
        if "WDC" in name:
            smoothed_source = ColumnDataSource(data=dict(x=wdc_d18o_ages, y=wdc_d18o_smoothed))
        elif "NGRIP" in name:
            smoothed_source = ColumnDataSource(data=dict(x=ngrip_d18o_ages, y=ngrip_d18o_smoothed))
        p.line('x', 'y', source=smoothed_source, line_width=2, color="red", legend_label="100 yr smoothed")
        p.legend.location = "top_left"

    plots.append(p)

plots[0].visible = True

checkbox_buttons = CheckboxButtonGroup(labels=list(data_series.keys()), active=[0], sizing_mode='scale_width')
callback = CustomJS(args=dict(plots=plots), code="""
    for (let i = 0; i < plots.length; i++) {
        plots[i].visible = this.active.includes(i);
    }
""")
checkbox_buttons.js_on_change('active', callback)

layout = column(checkbox_buttons, *plots, sizing_mode='stretch_both')
curdoc().add_root(layout)
output_file("interactive_plot_responsive.html")
show(layout)


In [ ]:
from bokeh.plotting import figure, show, output_file
from bokeh.layouts import column, row
from bokeh.models import CheckboxButtonGroup, CustomJS, HoverTool, ColumnDataSource, Div, Range1d
from bokeh.io import curdoc
import pandas as pd
import numpy as np


# Create a dictionary of data series
data_series = {
    "WDC CO2": df_wdc_co2,
    "WDC δ18O": df_wdc_d18o,
    "NGRIP δ18O": df_ngrip_d18o
}

# Create Bokeh plot for each series
plots = []
xlabels=['WD2014 Age (yr b1950)', 'WD2014 Age (yr b1950)', 'GICC05 Age (yr b2k)']
ylabels=['CO2 (ppm)', 'δ18O (per mil)', 'δ18O (per mil)']

i=0
for name, df in data_series.items():
    source = ColumnDataSource(data=dict(x=df.iloc[:, 0], y=df.iloc[:, 1]))
    p = figure(sizing_mode='stretch_both', title=name)

    p.line('x', 'y', source=source, line_width=2)
    p.visible = False  # Initially hide the plots
    # set x and y labels
    p.xaxis.axis_label = xlabels[i]
    p.yaxis.axis_label = ylabels[i]

    # invert the x-axis
    p.x_range.flipped = True
    p.x_range=Range1d(70000, 0)


    # Add HoverTool
    hover = HoverTool(tooltips=[("Time", "@x"), ("Value", "@y")])
    p.add_tools(hover)

    plots.append(p)
    i=i+1

# Initially show the first plot
plots[0].visible = True

# Create CheckboxButtonGroup with the first button active
checkbox_buttons = CheckboxButtonGroup(labels=list(data_series.keys()), active=[0], sizing_mode='scale_width')

# CustomJS callback to toggle visibility
callback = CustomJS(args=dict(plots=plots), code="""
    for (let i = 0; i < plots.length; i++) {
        plots[i].visible = this.active.includes(i);
    }
""")
checkbox_buttons.js_on_change('active', callback)

# Arrange plots and CheckboxButtonGroup in a column layout
layout = column(checkbox_buttons, *plots, sizing_mode='stretch_both')

# Set the layout for the current document
curdoc().add_root(layout)

# Output to HTML file
output_file("interactive_plot_responsive.html")

show(layout)  # Display the plot